In [4]:
from pytube import extract


def extract_video_id(url):
    """Extracts the YouTube video ID from a URL using pytube."""
    return extract.video_id(url)


# Example usage
url = "https://youtube.com/shorts/KV4D8MQrdhw?si=pgMOXp-e0P4XLq4m"
video_id = extract_video_id(url)
print(video_id)

KV4D8MQrdhw


In [5]:
from youtube_transcript_api import YouTubeTranscriptApi


def get_transcript(video_url):
    video_id = extract_video_id(video_url)
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
        for transcript in transcript_list:
            print(f"Language: {transcript.language}, Generated: {
                  transcript.is_generated}")
            transcript_data = transcript.fetch()
            return transcript_data
    except Exception as e:
        print(f"Error: {e}")


# Example usage
video_id = "https://www.youtube.com/watch?v=unpNp3Wi2Gk"
transcript = get_transcript(video_id)
transcript

Language: English (auto-generated), Generated: True


[{'text': "I was like damn he's really doing this",
  'start': 5.6,
  'duration': 5.63},
 {'text': 'that was awesome', 'start': 9.12, 'duration': 8.419},
 {'text': '[Music]', 'start': 11.23, 'duration': 8.99},
 {'text': 'yo what up folks Dr Mike here for',
  'start': 17.539,
  'duration': 5.381},
 {'text': 'Renaissance periodization you guys know',
  'start': 20.22,
  'duration': 4.559},
 {'text': "I have a pretty big Channel I'm sort of",
  'start': 22.92,
  'duration': 3.9},
 {'text': "important that thing is like I wasn't",
  'start': 24.779,
  'duration': 3.361},
 {'text': 'always like that I had a little tiny',
  'start': 26.82,
  'duration': 3.06},
 {'text': "Channel coming up we're always trying to",
  'start': 28.14,
  'duration': 4.68},
 {'text': 'give love to the guys coming up in their',
  'start': 29.88,
  'duration': 5.519},
 {'text': "own little way and it's my friend Jeff",
  'start': 32.82,
  'duration': 4.86},
 {'text': "nippard he's trying to get a YouTube",
  'start'

In [6]:
from openai import OpenAI
import json
from dotenv import load_dotenv
import os

load_dotenv()  # Load API key from .env file

# Use environment variable for API key
PERPLEXITY_API_KEY = os.getenv('PERPLEXITY_API_KEY')


def extract_workout_perplexity(transcript):
    if not transcript:
        print("Transcript is None, skipping Perplexity processing.")
        return None

    transcript_text = " ".join([entry["text"] for entry in transcript])

    prompt = f"""
    Extract workout details (exercise name, sets, reps, duration) from the following transcript:
    {transcript_text}

    Return just the result as a JSON list with this structure:
    [
        {{"exercise": "Push-ups", "sets": 3, "reps": 15, "duration": "30s"}},
        {{"exercise": "Squats", "sets": 4, "reps": 12, "duration": "40s"}}
    ]
    
    Please do not add anything else, simply return the JSON list. This includes adding markdown formatting.
    """

    messages = [{"role": "system", "content": "You are an assistant that extracts workout details. You will extract information related to exercises, sets, reps, and duration. You will only get information about exercises related to weightlifting or bodyweight exercises."},
                {"role": "user", "content": prompt}]

    try:
        client = OpenAI(api_key=PERPLEXITY_API_KEY,
                        base_url="https://api.perplexity.ai")
        response = client.chat.completions.create(
            model="sonar",
            messages=messages,
        )
    except Exception as e:
        print(f"Failed to connect to Perplexity API: {e}")
        return None

    # Extract JSON from Perplexity response
    extracted_data = response.choices[0].message.content
    print(extracted_data)
    try:
        return json.loads(extracted_data)
    except json.JSONDecodeError:
        print("Failed to parse Perplexity response as JSON.")
        return None


result = extract_workout_perplexity(transcript)
print(result)

[
    {"exercise": "Atlantis Incline Machine Press", "sets": 3, "reps": "8-15", "duration": "Not specified"},
    {"exercise": "Smith Machine Skull Crushers", "sets": 2, "reps": "11-12", "duration": "Not specified"},
    {"exercise": "Prime Chest Press", "sets": "1-2", "reps": "15-20", "duration": "Not specified"},
    {"exercise": "Overhead Extension", "sets": 1, "reps": "Not specified", "duration": "Not specified"}
]
[{'exercise': 'Atlantis Incline Machine Press', 'sets': 3, 'reps': '8-15', 'duration': 'Not specified'}, {'exercise': 'Smith Machine Skull Crushers', 'sets': 2, 'reps': '11-12', 'duration': 'Not specified'}, {'exercise': 'Prime Chest Press', 'sets': '1-2', 'reps': '15-20', 'duration': 'Not specified'}, {'exercise': 'Overhead Extension', 'sets': 1, 'reps': 'Not specified', 'duration': 'Not specified'}]


In [7]:
exercise_name = []
for exercise in result:
    exercise_name.append(exercise["exercise"])

exercise_name

['Atlantis Incline Machine Press',
 'Smith Machine Skull Crushers',
 'Prime Chest Press',
 'Overhead Extension']

In [9]:
import requests
import os
from dotenv import load_dotenv

# Load API Key from .env
load_dotenv()
# Ensure your .env file has HEVY_API_KEY=<your_api_key>
API_KEY = os.getenv("HEVY_API_KEY")


def get_exercise_templates(pages=5, page_size=100):
    base_url = "https://api.hevyapp.com/v1/exercise_templates"
    headers = {"accept": "application/json", "api-key": API_KEY}

    all_exercises = []

    for page in range(1, pages + 1):
        response = requests.get(f"{base_url}?page={page}&pageSize={
                                page_size}", headers=headers)

        if response.status_code == 200:
            data = response.json()
            print(data)
            # Adjust this key if needed
            exercises = data.get("exercise_templates", [])
            if not exercises:
                print(f"Page {page}: No exercises found.")
            all_exercises.extend(exercises)
        else:
            print(f"Error {response.status_code}: {response.text}")
            break  # Stop if an error occurs

    print(f"Total exercises retrieved: {len(all_exercises)}")
    return all_exercises


# Run function
exercises = get_exercise_templates()

{'page': 1, 'page_count': 5, 'exercise_templates': [{'id': '3BC06AD3', 'title': '21s Bicep Curl', 'type': 'weight_reps', 'primary_muscle_group': 'biceps', 'secondary_muscle_groups': [], 'equipment': 'barbell', 'is_custom': False}, {'id': 'B4F2FF72', 'title': 'Ab Scissors', 'type': 'reps_only', 'primary_muscle_group': 'abdominals', 'secondary_muscle_groups': [], 'equipment': 'none', 'is_custom': False}, {'id': '99D5F10E', 'title': 'Ab Wheel', 'type': 'reps_only', 'primary_muscle_group': 'abdominals', 'secondary_muscle_groups': [], 'equipment': 'other', 'is_custom': False}, {'id': '5E0DDACE', 'title': 'Aerobics', 'type': 'duration', 'primary_muscle_group': 'cardio', 'secondary_muscle_groups': [], 'equipment': 'none', 'is_custom': False}, {'id': '43573BB8', 'title': 'Air Bike', 'type': 'duration', 'primary_muscle_group': 'cardio', 'secondary_muscle_groups': [], 'equipment': 'machine', 'is_custom': False}, {'id': 'A69FF221', 'title': 'Arnold Press (Dumbbell)', 'type': 'weight_reps', 'prima

In [10]:
print(type(exercises))

<class 'list'>


In [11]:
import pandas as pd

df = pd.DataFrame(exercises)

df.head()

,id,title,type,primary_muscle_group,secondary_muscle_groups,equipment,is_custom
0,3BC06AD3,21s Bicep Curl,weight_reps,biceps,[],barbell,False
1,B4F2FF72,Ab Scissors,reps_only,abdominals,[],none,False
2,99D5F10E,Ab Wheel,reps_only,abdominals,[],other,False
3,5E0DDACE,Aerobics,duration,cardio,[],none,False
4,43573BB8,Air Bike,duration,cardio,[],machine,False


In [12]:
from fuzzywuzzy import process
# Function to find closest exercise in the DataFrame


def find_closest_exercise(exercise_name, df):
    closest_match = process.extractOne(exercise_name, df['title'])

    if closest_match[1] >= 80:  # If the similarity score is above 80
        # Find the id of the closest match
        closest_exercise = df[df['title'] == closest_match[0]].iloc[0]
        return closest_exercise['title'], closest_exercise['id']
    else:
        return 'None found'


# Search for closest matches
results = {}
for exercise in exercise_name:
    results[exercise] = find_closest_exercise(exercise, df)

# Display results
for exercise, result in results.items():
    print(f"Exercise: {exercise}")
    print(f"Closest Match: {result}")
    print()

/home/karan/.pyenv/versions/3.12.5/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Exercise: Atlantis Incline Machine Press
Closest Match: ('Bench Press (Barbell)', '79D0BB3A')

Exercise: Smith Machine Skull Crushers
Closest Match: ('Chest Fly (Machine)', '78683336')

Exercise: Prime Chest Press
Closest Match: ('Bench Press - Close Grip (Barbell)', '35B51B87')

Exercise: Overhead Extension
Closest Match: ('Back Extension (Hyperextension)', '4F5866F8')



In [13]:
import json

def convert_exercises(exercise_list, match_results, default_rest=90, default_notes="Stay slow and controlled."):
    exercises = []
    match_dict = {match[0]: match[1] for _, match in match_results.items()}
    
    for exercise in exercise_list:
        exercise_name = exercise['exercise']
        sets = exercise['sets']
        reps_list = list(map(str.strip, exercise['reps'].split(',')))  # Split reps if given as a range
        
        # Get closest match ID
        exercise_id = match_dict.get(exercise_name, "UNKNOWN")
        
        # Construct sets list
        sets_list = [{
            "type": "normal",
            "weight_kg": None,  # Default, since weight is not provided
            "reps": int(reps) if reps.isdigit() else reps,  # Convert to int if possible
            "distance_meters": None,
            "duration_seconds": None
        } for reps in reps_list]
        
        # Append formatted exercise
        exercises.append({
            "exercise_template_id": exercise_id,
            "superset_id": None,
            "rest_seconds": default_rest,
            "notes": default_notes,
            "sets": sets_list
        })
    
    return json.dumps({"exercises": exercises}, indent=2)

# Example usage
exercise_list = [
    {'exercise': 'Close Grip Barbell Incline Bench Press', 'sets': 3, 'reps': '8, 5, 15', 'duration': '3-4 minutes rest between sets'},
    {'exercise': 'Machine Shoulder Press', 'sets': 3, 'reps': '10-12', 'duration': 'Not specified'},
    {'exercise': 'Floor Reset Skull Crushers', 'sets': 3, 'reps': '6-8', 'duration': 'Not specified'},
    {'exercise': 'Bent Over Cable Peek Fly', 'sets': 3, 'reps': '10-12', 'duration': 'Not specified'},
    {'exercise': 'Machine Lateral Raises', 'sets': 3, 'reps': '20', 'duration': 'Not specified'},
    {'exercise': 'Plate Front Raises', 'sets': 2, 'reps': '15-20', 'duration': 'Not specified'},
    {'exercise': 'Diamond Push-ups', 'sets': 1, 'reps': 'To failure', 'duration': 'Not specified'}
]

match_results = {
    'Close Grip Barbell Incline Bench Press': ('Bench Press - Close Grip (Barbell)', '35B51B87'),
    'Machine Shoulder Press': ('Seated Shoulder Press (Machine)', '9237BAD1'),
    'Floor Reset Skull Crushers': ('Floor Triceps Dip', 'A57D38D5'),
    'Bent Over Cable Peek Fly': ('Cable Crunch', '23A48484'),
    'Machine Lateral Raises': ('Lateral Raise (Machine)', 'D5D0354D'),
    'Plate Front Raises': ('Plate Front Raise', 'DBF9273A'),
    'Diamond Push-ups': ('Diamond Push Up', '6575F52D')
}

formatted_json = convert_exercises(exercise_list, match_results)
print(formatted_json)

{
  "exercises": [
    {
      "exercise_template_id": "UNKNOWN",
      "superset_id": null,
      "rest_seconds": 90,
      "notes": "Stay slow and controlled.",
      "sets": [
        {
          "type": "normal",
          "weight_kg": null,
          "reps": 8,
          "distance_meters": null,
          "duration_seconds": null
        },
        {
          "type": "normal",
          "weight_kg": null,
          "reps": 5,
          "distance_meters": null,
          "duration_seconds": null
        },
        {
          "type": "normal",
          "weight_kg": null,
          "reps": 15,
          "distance_meters": null,
          "duration_seconds": null
        }
      ]
    },
    {
      "exercise_template_id": "UNKNOWN",
      "superset_id": null,
      "rest_seconds": 90,
      "notes": "Stay slow and controlled.",
      "sets": [
        {
          "type": "normal",
          "weight_kg": null,
          "reps": "10-12",
          "distance_meters": null,
          